In [9]:
import pyspark 
import time
from pyspark.sql import*
from pyspark import SparkContext, SparkConf

import pyspark.sql.functions as f
from pyspark.sql.functions import concat, concat_ws, lit, col, trim, length
from datetime import date

In [12]:
conf = SparkConf().setMaster("local").setAppName("verizon-wordcount")
#start spark cluster 
#if already started then get it else start it 
sc = SparkContext.getOrCreate(conf=conf)
#initialize SQLContext from spark cluster 
sqlContext = SQLContext(sc)

In [13]:
df = sqlContext.read.csv("./verizon_review.csv", header=True)

In [14]:
df.show(50)

+---+-----------------+--------------------+--------------------+--------------+----------------+
|_c0|           Device|               Title|          ReviewText|SubmissionTime|    UserNickname|
+---+-----------------+--------------------+--------------------+--------------+----------------+
|  0|Samsung Galaxy S7|Easy transition b...|Samsung has gone ...|  Mar 17, 2017|       datennant|
|  1|Samsung Galaxy S7|      Great upgrade!|The Galaxy S7 was...|  Mar 17, 2017|       Jules8613|
|  2|Samsung Galaxy S7|         Great phone|I love everything...|  Mar 16, 2017|missprettyncurvy|
|  3|Samsung Galaxy S7|     My New Galaxy 7|My friends told m...|  Mar 16, 2017|      Poohbear54|
|  4|Samsung Galaxy S7|Samsung and Veriz...|I have no photos ...|  Mar 03, 2017|          peg571|
|  5|Samsung Galaxy S7|Love my new Galax...|Going from a 7 ye...|  Mar 02, 2017|          DAGabe|
|  6|Samsung Galaxy S7|        Nice upgrade|Great working pho...|  Feb 27, 2017|         Sam8391|
|  7|Samsung Galaxy 

In [21]:
wordCountDF = df.withColumn('Text', concat(col('Title'), lit(" ") ,col('ReviewText'))) \
    .withColumn('word', f.explode(f.split(f.col('Text'), ' ')))\
    .groupBy('Device','word')\
    .count()\
    .filter(length('word')>3) \
    .sort('count', ascending=False)\
    .withColumn('Date', lit(date.today().strftime("%m/%d/%y"))) \
    .take(250)

In [22]:
wordCountDF

[Row(Device='Samsung Galaxy S7', word='phone', count=74, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='with', count=44, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='Galaxy', count=41, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='from', count=40, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='this', count=38, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='Samsung', count=36, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='phone.', count=35, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='Great', count=33, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='have', count=33, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='very', count=27, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='that', count=26, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='easy', count=22, Date='08/07/21'),
 Row(Device='Samsung Galaxy S7', word='Love', count=22, Date='08/07/21'),
 Row(Device='Samsung Galaxy S

In [23]:
hiveContext = pyspark.HiveContext(sc)